In [1]:
import sqlite3
from threading import Thread
import time

In [11]:
# Connexion à la base de données SQLite pour l'initialisation
init_conn = sqlite3.connect('example.db')
init_cursor = init_conn.cursor()

# Création de la table
init_cursor.execute('''
DROP TABLE IF  EXISTS comptes
''')

init_cursor.execute('''
CREATE TABLE IF NOT EXISTS comptes (
    id INTEGER PRIMARY KEY,
    solde INTEGER
)
''')

# Initialiser deux comptes pour le test
init_cursor.execute("INSERT INTO comptes (solde) VALUES (500)")
init_cursor.execute("INSERT INTO comptes (solde) VALUES (500)")
init_conn.commit()
init_conn.close()


In [12]:
def transaction_bancaire(compte_source, compte_dest, montant):
    # Créer une nouvelle connexion et un curseur pour ce thread
    conn = sqlite3.connect('example.db')
    cursor = conn.cursor()
    
    try:
        cursor.execute("BEGIN")
        
        # Récupérer le solde du compte source
        cursor.execute("SELECT solde FROM comptes WHERE id = ?", (compte_source,))
        solde_source = cursor.fetchone()[0]
        
        # Vérifier si le compte source a suffisamment de fonds
        if solde_source >= montant:
            # Débiter le compte source
            cursor.execute("UPDATE comptes SET solde = solde - ? WHERE id = ?", (montant, compte_source))
            # Simuler un délai pour accentuer les problèmes de concurrence
            time.sleep(0.1)
            # Créditer le compte destinataire
            cursor.execute("UPDATE comptes SET solde = solde + ? WHERE id = ?", (montant, compte_dest))
        else:
            print(f"Compte {compte_source} n'a pas assez de fonds.")
        
        conn.commit()
        print(f"Transféré {montant} du compte {compte_source} au compte {compte_dest}")
    except Exception as e:
        conn.rollback()
        print(f"Transaction échouée: {e}")
    finally:
        conn.close()


In [13]:
def concurrent_transactions():
    thread1 = Thread(target=transaction_bancaire, args=(1, 2, 100))
    thread2 = Thread(target=transaction_bancaire, args=(1, 2, 200))
    
    thread1.start()
    thread2.start()
    
    thread1.join()
    thread2.join()

# Appel de la fonction pour simuler les transactions concurrentes
concurrent_transactions()

Transaction échouée: database is locked
Transféré 200 du compte 1 au compte 2


In [14]:
def verification_soldes():
    conn = sqlite3.connect('example.db')
    cursor = conn.cursor()
    cursor.execute("SELECT * FROM comptes")
    comptes = cursor.fetchall()
    for compte in comptes:
        print(f"Compte {compte[0]}: Solde = {compte[1]}")
    conn.close()

# Appel de la fonction pour vérifier les soldes
verification_soldes()

Compte 1: Solde = 300
Compte 2: Solde = 700


### Interprétation des résultats
1. **Thread 2** a pu démarrer et obtenir le verrou pour effectuer sa transaction avant **Thread 1**.
2. Pendant que **Thread 2** exécutait sa transaction, **Thread 1** a essayé de démarrer une transaction mais n'a pas pu obtenir le verrou car **Thread 2** avait déjà verrouillé la base de données.
3. **Thread 2** a donc réussi à transférer 200 unités du compte 1 au compte 2.

### Pourquoi cela s'est-il produit ?

- **Ordre d'exécution et accès aux verrous** : Même si les threads sont lancés presque simultanément, le système d'exploitation et la gestion interne des verrous par SQLite détermineront quel thread obtient le verrou en premier.
- **Verrouillage de la base de données** : Une fois que **Thread 2** a obtenu le verrou, il a pu effectuer ses modifications sans interruption.
- **Échec du premier thread** : **Thread 1** a essayé d'accéder à la base de données pendant que **Thread 2** la modifiait, ce qui a entraîné l'échec de **Thread 1** avec l'erreur "database is locked".
